### Import packages

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_array,load_npz
from scipy import sparse

### Read data

In [2]:
df_val = pd.read_csv('df_val.csv')

In [3]:
R = load_npz('R_train.npz')

In [4]:
n_users,n_movies = R.shape

### Constructing accuracy function

In [5]:
one_idxs=[]
minus_one_idxs=[]
hold_out=[]
hold_out_minus=[]

In [6]:
for i in range(n_users):
    one_idxs.append(np.where(R[[i],:].toarray()[0] == 1)[0])
    minus_one_idxs.append(np.where(R[[i],:].toarray()[0] == -1)[0])
    hold_out.append(df_val.query(f"userId=={i} & rating==1").movieId.values)
    hold_out_minus.append(df_val.query(f"userId=={i} & rating==-1").movieId.values)

In [7]:
def accuracy_func(S,k=10):
    accuracy = []
    for i in range(n_users):
        output = S[i]
        np.put(output,one_idxs[i],-np.inf)
        np.put(output,minus_one_idxs[i],-np.inf)
        c = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out[i]))
        nc = len(np.intersect1d(np.argsort(output)[::-1][:k],hold_out_minus[i]))
#         acc = np.max([0,(c-nc)/(np.min([k,len(hold_out[i])+1]))]) ## Recal@K
        acc = np.max([0,(c-nc)/k]) ## HR@K
        accuracy.append(acc)
    return np.mean(accuracy)

### SVD function

In [8]:
def SVD(R,k=16):
    R = csr_array(R,dtype=np.float32)
    u,_,v = sparse.linalg.svds(R,k=k)
    S = np.matmul(u,v)
    return S

In [9]:
S = SVD(R,k=16)

In [10]:
accuracy_func(S)

0.27519867549668875